# 06_merge.ipynb

In [1]:
import pandas as pd
import geopandas as gpd
from dask.distributed import Client
import dask.dataframe as dd
from dask_jobqueue import SLURMCluster

import dask_geopandas
INPUT_DATA_EVICTIONS = "../../data/02_intermediate/evictions.csv"
INPUT_DATA_TRACTS = "../../data/02_intermediate/tracts.csv"
INPUT_DATA_TAX_PARCELS = "../../data/02_intermediate/tax_parcels.gpkg"
INPUT_DATA_ZESTIMATES = "../../data/02_intermediate/zestimates.csv"
INPUT_DATA_CRIME = "../../data/01_raw/crime_incidents"
OUTPUT_DATA_UNRESTRICTED = "../../data/03_cleaned/unrestricted.csv"
OUTPUT_DATA_ZILLOW = "../../data/03_cleaned/zestimates_analysis.csv"
OUTPUT_DATA_CRIME = "../../data/03_cleaned/crime_analysis.csv"
VERBOSE = True
N_JOBS = 10

/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_18590/933602976.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest

## 1. Loading Evictions Data

In [2]:
# Load evictions data.
with open(INPUT_DATA_EVICTIONS, 'r') as file:
    all_column_names = set(file.readline().replace("\"", "").replace("\n", "").split(","))
to_drop = {'Accuracy Score', 'Accuracy Type', 'Number', 'Street', 'Unit Type', 'Unit Number',
           'State', 'Zip', 'Country', 'Source', 'Census Year', 'State FIPS', 'County FIPS',
           'Place Name', 'Place FIPS', 'Census Tract Code', 'Census Block Code', 'Census Block Group',
           'Metro/Micro Statistical Area Code', 'Metro/Micro Statistical Area Type',
           'Combined Statistical Area Code', 'Metropolitan Division Area Code', 'court_location',
           'defendant', 'defendant_atty', 'defendant_atty_address_apt',
           'defendant_atty_address_city', 'defendant_atty_address_name', 'defendant_atty_address_state',
           'defendant_atty_address_street', 'defendant_atty_address_zip', 'docket_history', 'execution', 'judgment_for',
           'judgment_total', 'latest_docket_date', 'plaintiff', 'plaintiff_atty', 'plaintiff_atty_address_apt',
           'plaintiff_atty_address_city', 'plaintiff_atty_address_name', 'plaintiff_atty_address_state',
           'plaintiff_atty_address_street', 'plaintiff_atty_address_zip', 'Metropolitan Division Area Name',
           'property_address_city', 'property_address_state', 'property_address_street',
           'property_address_zip'}
df = pd.read_csv(INPUT_DATA_EVICTIONS, usecols=set(all_column_names) - set(to_drop))
original_N = len(df)
if VERBOSE:
    print(f"Beginning with {original_N} observations.")

# Drop cases missing file_date.
mask = df['file_date'].notna()
if VERBOSE:
    print(
        f"Dropping {(~mask).sum()} observations where file_date is missing ({100 * (((~mask).sum()) / original_N):.3} percent "
        f"of original dataset).")
df = df.loc[mask, :]

# Add file month and year to dataset.
df.loc[:, 'file_month'] = pd.to_datetime(df['file_date']).dt.strftime('%Y-%m')
df.loc[:, 'file_year'] = pd.to_datetime(df['file_date']).dt.year

# Clean the values in the judgment_for_pdu variable.
judgment_for_pdu_replacement_dict = {"unknown": "Unknown",
                                     "plaintiff": "Plaintiff",
                                     "defendant": "Defendant"}
df.loc[:, "judgment_for_pdu"] = (df.loc[:, "judgment_for_pdu"]
                                           .replace(judgment_for_pdu_replacement_dict))

# Replace missing values in money judgment column with zeroes.
df.loc[:, 'judgment'] = df['judgment'].fillna(0)

# Rename duration to case_duration.
df = df.rename(columns={'duration': 'case_duration'})

# Drop malformed addresses.
if VERBOSE:
    print(f"Dropping {df['property_address_full'].str.contains('span, span span').sum()} observations which "
          f"have malformed addresses "
          f"({df['property_address_full'].str.contains('span, span span').sum() / original_N:.2f} "
          f"percent of observations).")
df = df.loc[~df['property_address_full'].str.contains("span, span span"), :]

# Drop addresses without latitude and longitude.
if VERBOSE:
    print(f"Dropping {df[['longitude', 'latitude']].isna().any(axis=1).sum()} evictions missing latitude "
          f"or longitude ({df[['longitude', 'latitude']].isna().any(axis=1).sum() / original_N:.2f}) "
          f"percent of observations.")
df = df.dropna(subset=['longitude', 'latitude'])

Beginning with 40759 observations.
Dropping 0 observations where file_date is missing (0.0 percent of original dataset).
Dropping 24 observations which have malformed addresses (0.00 percent of observations).
Dropping 1 evictions missing latitude or longitude (0.00) percent of observations.


## 2. Merging Evictions With Census Tract Characteristics

In [3]:
# Merge with census tract characteristics.
df = df.rename(columns={'Full FIPS (tract)': 'tract_geoid'})
df = df.merge(pd.read_csv(INPUT_DATA_TRACTS, dtype={'tract_geoid': float}),
                                  on='tract_geoid',
                                  how='left',
                                  validate='m:1')
if VERBOSE:
    print(f"Successfully merged {df['med_hhinc2016'].notna().sum()} observations "
          f"({df['med_hhinc2016'].notna().sum() / original_N:.2f} percent of observations) with census "
          f"tracts.")

Successfully merged 40732 observations (1.00 percent of observations) with census tracts.


## 3. Merging Evictions With Zestimates

In [4]:
df = pd.read_csv(INPUT_DATA_ZESTIMATES).merge(df,
                                                     on='case_number',
                                                     how='right',
                                                     validate='1:1')
if VERBOSE:
    successfully_matched_observations = (~df['2022-12'].isna()).sum()
    print(
        f"Successfully matched {successfully_matched_observations} evictions "
        f"({100 * (successfully_matched_observations / len(df)) :.2f} percent of observations) to "
        f"Zestimates.")

# Rename columns containing Zestimates.
years = [str(year) for year in range(2013, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = ["2012-12"] + [str(year) + "-" + str(month) for year in years for month in months]
for value_var in value_vars:
    df = df.rename(columns={value_var: value_var + "_zestimate"})
value_vars_new = [value_var + "_zestimate" for value_var in value_vars]
value_vars = value_vars_new

Successfully matched 11496 evictions (28.22 percent of observations) to Zestimates.


## 4. Merging Evictions with Tax Parcels

In [16]:
# Request computing resources.
cluster = SLURMCluster(queue='batch',
                       cores=32,
                       memory='50 GB',
                       walltime='00:10:00',
                      scheduler_options={'dashboard_address': '8787'} )
cluster.scale(jobs=1)


/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38302 instead
  warnings.warn(


In [ ]:
df = gpd.GeoDataFrame(df,
                          geometry=gpd.points_from_xy(df['longitude'], df['latitude']))

df = df.set_crs("EPSG:4326", allow_override=True).to_crs('EPSG:26986')
with Client(cluster) as client:
    ddf = dask_geopandas.from_geopandas(df, npartitions=N_JOBS)

    tax_parcels_dgdf = dask_geopandas.read_file(INPUT_DATA_TAX_PARCELS, npartitions=N_JOBS, layer='layer')

    ddf = (dask_geopandas
                 .sjoin(ddf, tax_parcels_dgdf, how='inner', predicate='within')  # Only inner join is implemented.
                 .loc[ddf['LOC_ID'] != "F_819960_2934955", :]  # Drop the eviction which erroneously merges to two parcels.
                 .compute())  
if VERBOSE:
    successfully_matched_observations = ddf['LOC_ID'].notna().sum()
    print(f"Successfully matched {successfully_matched_observations} evictions "
          f"({100 * (successfully_matched_observations / original_N):.2f} percent of observations) to parcels."
          f"Dropping unmatcheed observations.")

## Merge Evictions With Crime Data

In [30]:
with Client(cluster) as client:
    crime_dgdf = dd.read_csv(INPUT_DATA_CRIME + "/*.csv")
    crime_dgdf = (dask_geopandas.from_dask_dataframe(crime_dgdf)
                  .dropna(subset=['Long', 'Lat', 'OCCURRED_ON_DATE'])
                  .rename(columns={'OCCURRED_ON_DATE': 'month_of_crime_incident'
# crime_gdf = (pd.concat([pd.read_csv(os.path.join(INPUT_DATA_CRIME, file)) for file in os.listdir(INPUT_DATA_CRIME)],
#                            axis=0)  # Read crime data.
#                  .dropna(subset=['Long', 'Lat', 'OCCURRED_ON_DATE'])
#                  .rename(columns={'OCCURRED_ON_DATE': 'month_of_crime_incident'}))
#     # Get month of each crime incident.
#     crime_gdf.loc[:, 'month_of_crime_incident'] = (pd.to_datetime(crime_gdf['month_of_crime_incident'].str[:10])
#                                                    .dt.to_period("M"))
#     # Convert crime data to a GeoDataFrame.
#     crime_gdf = (gpd
#                  .GeoDataFrame(crime_gdf,
#                                geometry=gpd.points_from_xy(crime_gdf['Long'], crime_gdf['Lat']))
#                  .set_crs("EPSG:4326")
#                  .to_crs("EPSG:26986")
#                  .drop(columns=['Lat', 'Long', 'Location', 'UCR_PART', 'HOUR', 'DAY_OF_WEEK', 'YEAR', 'MONTH',
#                                'REPORTING_AREA', 'DISTRICT', 'STREET', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION']))
#     crime_gdf = crime_gdf.iloc[:100]
#     crime_gdf = dask_geopandas.from_geopandas(crime_gdf, npartitions=4)

#     # Spatial join of crimes and evictions
#     join = dask_geopandas.sjoin(crime_gdf,
#                                 eviction_gdf,
#                                 how='inner',
#                                 predicate='within')
#     evictions_matched_with_crimes = join.compute()
#     evictions_matched_with_crimes.to_csv('~/Desktop/parcel_join_results.csv')

Dask DataFrame Structure:
              INCIDENT_NUMBER OFFENSE_CODE OFFENSE_CODE_GROUP OFFENSE_DESCRIPTION DISTRICT REPORTING_AREA SHOOTING OCCURRED_ON_DATE   YEAR  MONTH DAY_OF_WEEK   HOUR UCR_PART  STREET      Lat     Long Location
npartitions=9                                                                                                                                                                                                   
                       object        int64             object              object   object          int64  float64           object  int64  int64      object  int64   object  object  float64  float64   object
                          ...          ...                ...                 ...      ...            ...      ...              ...    ...    ...         ...    ...      ...     ...      ...      ...      ...
...                       ...          ...                ...                 ...      ...            ...      ...              ...    ...